In [37]:
import sys
sys.path.append("..")

from galpy.potential import evaluatePotentials, MiyamotoNagaiPotential, NFWPotential, HernquistPotential

import numpy as np
from scipy import stats
from IPython.display import Image, display

import corner

import datetime
import random

import copy

from areposnap.gadget import gadget_readsnap
from areposnap.gadget_subfind import load_subfind

from auriga_basics import *

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib import rcParams

import datetime
%matplotlib inline

rcParams['font.size'] = 16
rcParams['lines.linewidth'] = 1.5
rcParams['axes.linewidth'] = 1.5
rcParams['ytick.major.width'] = 1.5
rcParams['xtick.major.width'] = 1.5
rcParams['image.cmap'] = 'magma'
rcParams['savefig.bbox'] = 'tight'
rcParams['savefig.dpi'] = 300
rcParams['savefig.format'] = 'png'

cmap = copy.copy(plt.cm.get_cmap(rcParams['image.cmap']))
cmap.set_bad((0,0,0))
%config InlineBackend.figure_format = 'retina'


from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
machine = 'mac'
#machine = 'magny'
machine = 'virgo'

if machine == 'magny':
    basedir = "/hits/universe/GigaGalaxy/level4_MHD/"
    filedir = "/home/extmilan/masterthesis/files/"
    plotdir = "/home/extmilan/masterthesis/plots/"
elif machine == 'mac': 
    basedir = "/Users/smilanov/Desktop/Auriga/level4/"
    filedir = "/Users/smilanov/Documents/masterthesis/auriga_files/files/"
    plotdir = "/Users/smilanov/Documents/masterthesis/auriga_files/plots/"
elif machine == 'virgo': 
    basedir = "/virgo/simulations/Auriga/level4_MHD/"
    filedir = "/u/milas/masterthesis/masterproject/files/"
    plotdir = "/u/milas/masterthesis/masterproject/plots/"
   

In [77]:
def energy(v, R = None, z = None, pot_au = None, pot_galpy = None, evaluate_galpy_pot = True, use_auriga_pot = False, R0_kpc = 8., v0_kms = 220.):
    if use_auriga_pot == True:
        pot = pot_au
        if len(pot) != np.shape(v)[1]:
            print('Input for potential and for velocities has different length. Please check')
            print(len(pot), np.shape(v))
    elif evaluate_galpy_pot == True:
        if isinstance(R, np.ndarray):
            pot = np.array([evaluatePotentials(pot_galpy, R[ii]/R0_kpc, z[ii]/R0_kpc) for ii in range(len(R))])* v0_kms**2
        else:
            print('Potential should be evaluated but R does not seem to be an aray. Please check.')
            print(R)
    else: print('Please specify potential.')
        
    term1 = 0.5 * np.sum(v**2, axis =0)    
    term2 = pot
    e = term1 + term2
    return(e)

def angmom(R, vT):
    lz = R * vT
    return(lz)

In [64]:
def setup_galpy_potential(a_MND_kpc, b_MND_kpc, a_NFWH_kpc, a_HB_kpc, n_MND, n_NFWH, n_HB, _REFR0_kpc):
    
    #test input:
    if (a_MND_kpc <= 0.) or (b_MND_kpc <= 0.) or (a_NFWH_kpc <= 0.) or (a_HB_kpc <= 0.) \
       or (n_MND <= 0.) or (n_NFWH <= 0.) or (n_HB <= 0.) or (n_MND >= 1.) or (n_NFWH >= 1.) or (n_HB >= 1.):
        raise ValueError('Error in setup_galpy_potential: '+\
                         'The input parameters for the scaling profiles do not correspond to a physical potential.')
    if np.fabs(n_MND + n_NFWH + n_HB - 1.) > 2e-2:
        raise ValueError('Error in setup_galpy_potential: '+\
                         'The sum of the normalization does not add up to 1.')
        
    #trafo to galpy units:
    a_MND  = a_MND_kpc  / _REFR0_kpc
    b_MND  = b_MND_kpc  / _REFR0_kpc
    a_NFWH = a_NFWH_kpc / _REFR0_kpc
    a_HB   = a_HB_kpc   / _REFR0_kpc
    
    #setup potential:
    disk = MiyamotoNagaiPotential(
                a = a_MND,
                b = b_MND,
                normalize = n_MND)
    halo = NFWPotential(
                a = a_NFWH,
                normalize = n_NFWH)
    bulge = HernquistPotential(
                a = a_HB,
                normalize = n_HB) 
     
    return [disk,halo,bulge]

In [65]:
# load potential parameters
R0_kpc_all, v0_kms_all, a_MND_kpc_all, b_MND_kpc_all, a_NFWH_kpc_all, a_HB_kpc_all, v0_MND_kms_all, v0_NFWH_kms_all, v0_HB_kms_all = np.loadtxt(filedir + 'potential/fitted_potential_params_dec18.txt')

n_MND_all  = v0_MND_kms_all**2  / v0_kms_all**2
n_NFWH_all = v0_NFWH_kms_all**2 / v0_kms_all**2
n_HB_all   = v0_HB_kms_all**2   / v0_kms_all**2

In [97]:
level = 4

startnr = 110
endnr   = 128

colors = ['black', 'black', 'xkcd:hot pink', 'xkcd:indigo blue', 'xkcd:saffron']

IDs_all      = []
angmom_all   = []
e_galpy_all  = []
e_auriga_all = []
for prognr in [2]: #2, 3, 4
    IDs = np.loadtxt(filedir + 'auriga/surviving_GC_zsignchange_IDs_prog{}.txt'.format(prognr))
    progcolor = colors[prognr]
    
    for halo_number in [24]:  # range(1, 31):
        halodir = basedir+"halo_{0}/".format(halo_number)
        snappath = halodir+"output/"

        for snapnr in range(startnr,endnr, 1):
            i = snapnr - 60
            print('Snapindex: {0}'.format(i))
            print("level    : {0}".format(level))
            print("halo     : {0}".format(halo_number))
            print("snapnr   : {0}".format(snapnr))
            print("basedir  : {0}".format(basedir))
            print("halodir  : {0}".format(halodir))
            print("snappath : {0}\n".format(snappath))
            s, sf = eat_snap_and_fof(level, halo_number, snapnr, snappath, loadonlytype=[4], 
                haloid=0, galradfac=0.1, verbose=True) 

            # Clean negative and zero values of gmet to avoid RuntimeErrors
            # later on (e.g. dividing by zero)
            s.data['gmet'] = np.maximum( s.data['gmet'], 1e-40 )

            gcmask = np.isin(s.id, IDs)
            print('Number of GCs: {}.'.format(np.sum(gcmask)))

            (R_kpc, phi_rad, z_kpc), (vR_kms, vT_kms, vz_kms) = get_cylindrical_vectors(s, sf, gcmask)
            pot_au = s.pot[gcmask]

            print('Sum of normalizations after fitting potential parameters: {}.'.format(np.sum([n_MND[i], n_NFWH[i], n_HB[i]])))
        

            pot_galpy = setup_galpy_potential(a_MND_kpc_all[i], b_MND_kpc_all[i], a_NFWH_kpc_all[i], a_HB_kpc_all[i], n_MND[i], n_NFWH[i], n_HB[i], R0_kpc_all[i])

            # this would mean that there are no merged GCs
            if np.sum(gcmask) == 0:
                print("Skipped snap {} since there are no merged GCs.".format(snapnr))
                #continue
                
            vel_arr_kms = np.array([vR_kms, vT_kms, vz_kms])
            
            angmom_arr_kpckms = angmom(R_kpc, vT_kms)

            e_galpy_km2s2 = energy(vel_arr_kms, R_kpc, z_kpc, pot_galpy = pot_galpy, R0_kpc = R0_kpc_all[i], v0_kms = v0_kms_all[i])
            e_au_km2s2 = energy(vel_arr_kms, pot_au = pot_au, use_auriga_pot=True, evaluate_galpy_pot=False)
            print('Galpy energy:',np.mean(e_galpy_km2s2), np.std(e_galpy_km2s2))
            print('Auriga energy:',np.mean(e_au_km2s2), np.std(e_au_km2s2))
            
            IDs_saved = s.id[gcmask]
            savedata = np.array([IDs_saved, angmom_arr_kpckms, e_au_km2s2, e_galpy_km2s2])
            header = 'IDs, angmom_arr_kpckms, e_au_km2s2, e_galpy_km2s2'
            np.savetxt(filedir + 'potential/progenitor{}/energy_angmom_dec18.txt', savedata, header = header) 
            
            fig, ax = plt.subplots()
            h = ax.hist2d(angmom_arr_kpckms, e_galpy_km2s2, norm = LogNorm(), cmap = cmap, range=[[np.min(angmom_arr_kpckms), np.max(angmom_arr_kpckms)], [-140000, -60000]], bins = 101)
            ax.set_xlabel('L$_\mathrm{z}$ [kpc km s$^{-1}$]')
            ax.set_ylabel('E [km $^2$ s$^{-2}$]')
            plt.colorbar(h[3])
            plt.show()     
            
            fig, ax = plt.subplots()
            h = ax.hist2d(angmom_arr_kpckms, e_au_km2s2,norm = LogNorm(), cmap = cmap, range=[[np.min(angmom_arr_kpckms), np.max(angmom_arr_kpckms)], [-140000, -60000]], bins = 101)
            ax.set_xlabel('L$_\mathrm{z}$ [kpc km s$^{-1}$]')
            ax.set_ylabel('E [km $^2$ s$^{-2}$]')
            plt.colorbar(h[3])
            plt.show()

Snapindex: 50
level    : 4
halo     : 24
snapnr   : 110
basedir  : /virgo/simulations/Auriga/level4_MHD/
halodir  : /virgo/simulations/Auriga/level4_MHD/halo_24/
snappath : /virgo/simulations/Auriga/level4_MHD/halo_24/output/

Found 1596516 stars.
Rotated pos.
Rotated vel.

galrad  : 0.022285720705986025
redshift: 0.22598838626019768
time    : 0.8156684118765909
center  : [ 0.  0.  0.]

Number of GCs: 6046.
Sum of normalizations after fitting potential parameters: 0.9958463354656286.
Galpy energy: -86386.958775 12968.420443
Auriga energy: -89060.6872862 10770.2773819


TypeError: 'numpy.ndarray' object is not callable

In [95]:
savedata = np.array([IDs_all, angmom_all, e_auriga_all, e_galpy_all])
np.savetxt(filedir + 'potential/progenitor{}/energy_angmom_dec18.txt', savedata) 

TypeError: Mismatch between array dtype ('float64') and format specifier ('%.18e %.18e %.18e %.18e %.18e %.18e %.18e %.18e %.18e %.18e %.18e %.18e %.18e %.18e %.18e %.18e %.18e %.18e')

In [91]:
np.shape(IDs_all)

(18, 6046)

In [94]:
np.shape(e_galpy_all)

(18, 6046)